In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.memory import ConversationKGMemory

In [2]:
llm = ChatOpenAI(
  model_name="gpt-4",
  openai_api_key=openai_api_key,
  temperature=0.1
)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

In [3]:
examples = [
    {"input": "탑건", "output": "🛩️👨‍✈️🔥"},
    {"input": "대부", "output": "👨‍👨‍👦🔫🍝"},
    {"input": "타이타닉", "output": "🚢💑💔"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="영화 제목: {input}\n이모티콘: {output}"
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    prefix="당신은 영화 제목을 이모지 세 개로 변환하는 AI입니다. 항상 정확히 세 개의 이모지만 사용하세요. 다음은 영화 제목을 이모티콘으로 변환한 예시입니다:",
    suffix="영화 제목: {input}\n이모티콘:",
    input_variables=["input"]
)

In [4]:
chain = prompt | llm

def invoke_chain(question):
    result = chain.invoke({"input": question})

    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    
    print(result.content)

In [9]:
invoke_chain("캐스트어웨이")

✈️🏝️📦


In [20]:
invoke_chain("E.T")

👽🚲🌕
